In [ ]:
# !pip install peft

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import AutoPeftModelForCausalLM, PeftConfig, PeftModel
import torch

In [ ]:
lora_repo = "ArjunRavi/mental-health-finetune"
base_repo = "NousResearch/Llama-2-7b-chat-hf"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_repo)

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    base_repo,
    device_map="auto",
    dtype=torch.bfloat16
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [ ]:
lora_model = PeftModel.from_pretrained(base_model, lora_repo)

adapter_config.json:   0%|          | 0.00/785 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

In [ ]:
lora_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): 

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_repo)

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [ ]:
prompt = "I am feeling very down, without any reason. Also I am very distracted and cannot find interest in anything."

In [ ]:
input = tokenizer(prompt,return_tensors='pt').to('cuda')

In [ ]:
input

{'input_ids': tensor([[    1,   306,   626, 11223,  1407,  1623, 29892,  1728,   738,  2769,
         29889,  3115,   306,   626,  1407,  1320,  1461,   287,   322,  2609,
          1284,  4066,   297,  3099, 29889]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]], device='cuda:0')}

In [ ]:
response = lora_model.generate(**input,max_new_tokens=150,temperature=0.3)

In [ ]:
len(response[0])

175

In [ ]:
response

tensor([[    1,   306,   626, 11223,  1407,  1623, 29892,  1728,   738,  2769,
         29889,  3115,   306,   626,  1407,  1320,  1461,   287,   322,  2609,
          1284,  4066,   297,  3099, 29889,   306,   884,   505,   263,  3287,
           310, 13133,   393,   306,  2609,  2761, 29889,   306,   626,   885,
          1965,   393,   306,   626, 19035,   590,  3458, 29889,  1724,   508,
           306,   437, 29973,    13,  3112, 10083,   763,   366,  1122,   367,
         10623,  3277,   777, 25828,  4835,   310,   316,  2590, 29889,   897,
          2590,   338,   263,  3619, 19119,  9045,   766,  2098,   393,   508,
          6602,  5019, 17126,   310,  5046, 29892, 23346, 29892,   470, 11314,
          7823,   537, 29889,   739,   338,  4100,   304,  6456,   393,   316,
          2590,   338,   263,  7539,   519,  4195, 29892,   322,   727,   526,
          1784,  1422, 14502,  3987,  3625, 29889,    13,  3644,   366,   526,
         11223,   975,  1332,   295,  2168,   470,  

In [ ]:
prompt

'I am feeling very down, without any reason. Also I am very distracted and cannot find interest in anything.'

In [ ]:
tokenizer.decode(response[0])

'<s> I am feeling very down, without any reason. Also I am very distracted and cannot find interest in anything. I am also having trouble sleeping. What can I do?\nIt sounds like you may be experiencing some symptoms of depression. Depression is a common mental health condition that can affect anyone regardless of age, gender, or background. It can cause a range of symptoms including feeling sad, hopeless, and helpless, as well as difficulty concentrating, sleeping, and enjoying activities that you used to enjoy.\nIf you are experiencing these symptoms, it is important to seek help from a mental health professional. They can help you identify the underlying causes of your depression and develop a treatment plan that is tailored to your needs.\nIn the meantime, here are some things you can try'

In [ ]:
prompt = """<s>[INST] <<SYS>>
You are a helpful and empathetic assistant.
<</SYS>>

I am feeling very down, without any reason. Also I am very distracted and cannot find interest in anything.
[/INST]"""

input = tokenizer(prompt, return_tensors="pt").to("cuda")


In [ ]:
input

{'input_ids': tensor([[    1,     1, 29961, 25580, 29962,  3532, 14816, 29903,  6778,    13,
          3492,   526,   263,  8444,   322,   953,  2084,  7492, 20255, 29889,
            13, 29966,   829, 14816, 29903,  6778,    13,    13, 29902,   626,
         11223,  1407,  1623, 29892,  1728,   738,  2769, 29889,  3115,   306,
           626,  1407,  1320,  1461,   287,   322,  2609,  1284,  4066,   297,
          3099, 29889,    13, 29961, 29914, 25580, 29962]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [ ]:
response = lora_model.generate(**input, max_new_tokens=250, temperature=0.3, pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(response[0], skip_special_tokens=True))

[INST] <<SYS>>
You are a helpful and empathetic assistant.
<</SYS>>

I am feeling very down, without any reason. Also I am very distracted and cannot find interest in anything.
[/INST]  Sorry to hear that you're feeling down and distracted. It's important to remember that you don't have to have a reason to feel down or distracted. Sometimes our emotions can be unpredictable and can change without us even realizing it.

It's great that you're aware of how you're feeling and that you're looking for ways to improve your emotional state. Here are some suggestions that may help:

1. Practice self-care: Take care of yourself by doing things that make you feel good. This can include things like getting enough sleep, eating healthy foods, exercising, and doing activities that you enjoy.
2. Connect with others: Talk to a friend or family member about how you're feeling. Sometimes just talking about your emotions can help you feel better.
3. Try something new: Try something new that you've never

In [ ]:
with torch.no_grad():
  outputs = lora_model(**input)
  logits = outputs.logits
print("Logits shape:", logits.shape)  # [batch_size, seq_len, vocab_size]



Logits shape: torch.Size([1, 57, 32000])


In [ ]:
predicted_token = torch.argmax(logits[:, -1, :], dim=-1)  # shape [batch_size]
predicted_token

tensor([29871], device='cuda:0')

In [ ]:
predicted_token = predicted_token.unsqueeze(1)  # shape [1, 1]

In [ ]:
predicted_token.shape,input['input_ids'].shape

(torch.Size([1, 1]), torch.Size([1, 57]))

In [ ]:
input['input_ids']

tensor([[    1,     1, 29961, 25580, 29962,  3532, 14816, 29903,  6778,    13,
          3492,   526,   263,  8444,   322,   953,  2084,  7492, 20255, 29889,
            13, 29966,   829, 14816, 29903,  6778,    13,    13, 29902,   626,
         11223,  1407,  1623, 29892,  1728,   738,  2769, 29889,  3115,   306,
           626,  1407,  1320,  1461,   287,   322,  2609,  1284,  4066,   297,
          3099, 29889,    13, 29961, 29914, 25580, 29962]], device='cuda:0')

In [ ]:
torch.cat([input['input_ids'],predicted_token],dim=-1)

tensor([[    1,     1, 29961, 25580, 29962,  3532, 14816, 29903,  6778,    13,
          3492,   526,   263,  8444,   322,   953,  2084,  7492, 20255, 29889,
            13, 29966,   829, 14816, 29903,  6778,    13,    13, 29902,   626,
         11223,  1407,  1623, 29892,  1728,   738,  2769, 29889,  3115,   306,
           626,  1407,  1320,  1461,   287,   322,  2609,  1284,  4066,   297,
          3099, 29889,    13, 29961, 29914, 25580, 29962, 29871]],
       device='cuda:0')

In [ ]:
input_ids = input['input_ids']
predictions = []
for i in range(10):
  with torch.no_grad():
    outputs = lora_model(input_ids)
    logits = outputs.logits

  predicted_token = torch.argmax(logits[:,-1,:],dim=-1)
  predictions.append(predicted_token)
  predicted_token = predicted_token.unsqueeze(1)
  input_ids = torch.cat([input_ids,predicted_token],dim=-1)

In [ ]:
input_ids[0][-10]

tensor(29871, device='cuda:0')

In [ ]:
generated_text = []
for i in range(-10,0):
  decode_token = tokenizer.decode(input_ids[0][i])
  generated_text.append(decode_token.strip())
generated_text

['', 'Sorry', 'to', 'hear', 'that', 'you', "'", 're', 'feeling', 'down']

In [ ]:
generated_text_str = " ".join(generated_text)
generated_text_str

" Sorry to hear that you ' re feeling down"

In [ ]:
prediction = torch.argmax(logits[:,:,:],dim=-1)

In [ ]:
prediction

tensor([[  835,   835,  1792, 29900,  1724,  4721, 29899,  4590,   306,    13,
           526,   263, 11643,  2022,  1871,  2084,  7492,  2022, 29889,   306,
         29902, 29925, 14816, 29903, 29901,     2, 29902, 29902,   505,   263,
          1407,  1623,   322,   322,   263, 17385, 29889,   306, 29892,   626,
          2534, 24937,  1461,   287,   322,   306,  8569,   590,   297,  3099,
         29889,   306,    13, 11889, 11889,     2, 29871]], device='cuda:0')

In [ ]:
predicted_prompt = []
for pred in prediction[0]:
  pred_token = tokenizer.decode(pred)
  predicted_prompt.append(pred_token)

In [ ]:
predicted_prompt_str = " ".join(predicted_prompt)
predicted_prompt_str

'### ### user 0 What Pre - OP I \n are a psych person inform path etic person . I I P SY S : </s> I I have a very down and and a motiv . I , am having anxious ract ed and I focus my in anything . I \n USER USER </s> '